# Working with BBM database

This notebook illustrates the capability of AEON.py to fetch and prepare BN models directly from the Biodivine Boolean Models database.

*Work in progress...*

In [1]:
from biodivine_aeon import *

You can retrieve model IDs from the BBM API or query for a BN with a given ID. 

At the moment, the model ID is a string ID used by the API, which is not the same as the numerical id clasically used in the BBM database. This is a temporary solution until the BBM endpoint is updated to provide the numerical id as well.

### Retrieving and filtering model IDs

Let's first check out how we can request and filter model IDs.

You can either retrieve all IDs in the database, or filter them according to number of variables, inputs, regulations, or keywords. Let's check both variants.

In [2]:
ids = BiodivineBooleanModels.fetch_ids()
print(len(ids))  # Show the total number of models
ids[:5]  # Show the first 5 ids

230


['202fec2d-a27d-45cd-a1d2-6d619412b1d5',
 '15ce1709-b98e-4fcf-9183-9adda842a437',
 'dad447c8-c14d-40d2-8496-d8953690a20a',
 '6454b139-57fa-4a7f-8f52-5623db697dc5',
 '6bfb7209-57ea-40e1-9d78-cbde5f9d106f']

In [3]:
# We can configure the filtering using the BbmFilterConfig instance
config = BbmFilterConfig()
print(f"Empty config:\n\t{config}")

config["min_variables"] = 10
config["max_variables"] = 20
config["keywords"] = ["cell-collective", "curated"]
print(f"Initialized config:\n\t{config}")

Empty config:
	BbmFilterConfig(variables=<None,None>, inputs=<None,None>, regulations=<None,None>, keywords=[])
Initialized config:
	BbmFilterConfig(variables=<10,20>, inputs=<None,None>, regulations=<None,None>, keywords=["cell-collective", "curated"])


In [4]:
ids = BiodivineBooleanModels.fetch_ids(config)
print(len(ids))  # Show the total number of models
ids[:5]  # Show the first 5 ids

29


['dad447c8-c14d-40d2-8496-d8953690a20a',
 '36dca970-8bea-4620-9088-8da2890211a4',
 '86248135-ed19-44ea-a13b-daf0ca504986',
 '02329d2b-8da8-4c97-86ae-743ad3f654ec',
 '20a7ff67-dc62-4240-b2ac-6aa12382d4a0']

### Retrieving particular networks

Let's now retrieve a particular Boolean network using its ID. You can fetch the BN as is, or use the function arguments to adjust some network settings, such as how the inputs are handled.

In [5]:
# Fetch a small network (with 18 variables, 8 of which are inputs) by its ID
bn = BiodivineBooleanModels.fetch_network("b427a7d3-5158-48ab-991b-1db895dd2b4d")
print(bn)

BooleanNetwork(variables=18, regulations=18, explicit_parameters=0, implicit_parameters=8)


In [6]:
# Fetch the same network, but with inlined inputs
bn = BiodivineBooleanModels.fetch_network("b427a7d3-5158-48ab-991b-1db895dd2b4d", inline_inputs=True)
print(bn)

BooleanNetwork(variables=10, regulations=10, explicit_parameters=8, implicit_parameters=0)


### Retrieving models with metadata

We can also retrieve a more general `BbmModel` instance that contains not only the network data, but also additional metadata. The Boolean network can then be easily extracted from this object.

In [7]:
model = BiodivineBooleanModels.fetch_model("b427a7d3-5158-48ab-991b-1db895dd2b4d")
print(model)

BbmModel(id=b427a7d3-5158-48ab-991b-1db895dd2b4d, name=VEGF-PATHWAY-OF-DROSOPHILA, variables=10, inputs=8, regulations=18)


You can select a particular method, depending whether you want to leave the model inputs free, or fix them.

In [8]:
bn = model.to_bn_default()
print(bn.to_aeon())

v_Pvr -> v_Drk
v_Raf -> v_Dsor1
v_CNK -> v_Raf
v_Ksr -> v_Raf
v_Src42 -> v_Raf
v_Ras -> v_Raf
v_Rl -> v_Pnt
v_Drk -> v_Sos
v_Pvf2 -> v_Pvr
v_Pvf3 -> v_Pvr
v_Pvf1 -> v_Pvr
v_Dsor1 -> v_Rl
v_msk -> v_Rl
v_Sty -| v_Ras
v_Sos -> v_Ras
v_Aop -| v_Targets
v_Pnt -> v_Targets
v_Rl -| v_Aop
$v_Aop: !v_Rl
$v_Drk: v_Pvr
$v_Dsor1: v_Raf
$v_Pnt: v_Rl
$v_Pvr: v_Pvf2 | v_Pvf1 | v_Pvf3
$v_Raf: v_Ras & v_CNK & v_Ksr & v_Src42
$v_Ras: v_Sos & !v_Sty
$v_Rl: v_Dsor1 & v_msk
$v_Sos: v_Drk
$v_Targets: v_Pnt & !v_Aop



In [9]:
bn = model.to_bn_inputs_true()
print(bn.to_aeon())

v_Pvr -> v_Drk
v_Raf -> v_Dsor1
v_CNK -> v_Raf
v_Ksr -> v_Raf
v_Src42 -> v_Raf
v_Ras -> v_Raf
v_Rl -> v_Pnt
v_Drk -> v_Sos
v_Pvf2 -> v_Pvr
v_Pvf3 -> v_Pvr
v_Pvf1 -> v_Pvr
v_Dsor1 -> v_Rl
v_msk -> v_Rl
v_Sty -| v_Ras
v_Sos -> v_Ras
v_Aop -| v_Targets
v_Pnt -> v_Targets
v_Rl -| v_Aop
$v_Aop: !v_Rl
$v_CNK: true
$v_Drk: v_Pvr
$v_Dsor1: v_Raf
$v_Ksr: true
$v_Pnt: v_Rl
$v_Pvf1: true
$v_Pvf2: true
$v_Pvf3: true
$v_Pvr: v_Pvf2 | v_Pvf1 | v_Pvf3
$v_Raf: v_Ras & v_CNK & v_Ksr & v_Src42
$v_Ras: v_Sos & !v_Sty
$v_Rl: v_Dsor1 & v_msk
$v_Sos: v_Drk
$v_Src42: true
$v_Sty: true
$v_Targets: v_Pnt & !v_Aop
$v_msk: true

